In [2]:
!wget https://he-s3.s3.amazonaws.com/media/hackathon/machine-learning-challenge-3/predict-ad-clicks/205e1808-6-dataset.zip

--2020-02-21 05:08:58--  https://he-s3.s3.amazonaws.com/media/hackathon/machine-learning-challenge-3/predict-ad-clicks/205e1808-6-dataset.zip
Resolving he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)... 52.219.48.36
Connecting to he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)|52.219.48.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 406223751 (387M) [application/zip]
Saving to: ‘205e1808-6-dataset.zip’

205e1808-6-dataset. 100%[===================>] 387.40M  60.8MB/s    in 6.7s    

2020-02-21 05:09:10 (57.9 MB/s) - ‘205e1808-6-dataset.zip’ saved [406223751/406223751]



In [3]:
!unzip 205e1808-6-dataset.zip

Archive:  205e1808-6-dataset.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [4]:
!ls

205e1808-6-dataset.zip	sample_data  sample_submission.csv  test.csv  train.csv


In [0]:
import pandas as pd

In [0]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [7]:
print(len(train), len(test))

12137810 3706907


In [8]:
train.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click
0,IDsrk7SoW,2017-01-14 09:42:09,4709696.0,887235,17714,20301556,e,Firefox,NaN,0
1,IDmMSxHur,2017-01-18 17:50:53,5189467.0,178235,21407,9434818,b,Mozilla Firefox,Desktop,0
2,IDVLNN0Ut,2017-01-11 12:46:49,98480.0,518539,25085,2050923,a,Edge,NaN,0
3,ID32T6wwQ,2017-01-17 10:18:43,8896401.0,390352,40339,72089744,c,Firefox,Mobile,0
4,IDqUShzMg,2017-01-14 16:02:33,5635120.0,472937,12052,39507200,d,Mozilla Firefox,Desktop,0


In [9]:
test.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid
0,IDFDJVI,2017-01-22 09:55:48,755610.0,808980,17714,26391770,b,Mozilla Firefox,Desktop
1,IDNWkTQ,2017-01-22 03:54:39,3714899.0,280355,12052,39507200,b,Edge,Tablet
2,ID9pRmM,2017-01-21 10:25:50,4378333.0,930819,30580,46148550,e,Mozilla Firefox,NaN
3,IDHaQaj,2017-01-22 14:45:53,1754730.0,612234,11837,8837581,b,Edge,Tablet
4,IDT2CrF,2017-01-22 09:34:07,5299909.0,524289,45620,31388981,b,Mozilla,NaN


In [10]:
train.isnull().sum(axis=0)

ID                   0
datetime             0
siteid         1212515
offerid              0
category             0
merchant             0
countrycode          0
browserid       608327
devid          1820299
click                0
dtype: int64

In [11]:
train['siteid'].unique(), train['browserid'].unique(), train['devid'].unique()

(array([4709696., 5189467.,   98480., ..., 4086584.,  762989., 4284843.]),
 array(['Firefox', 'Mozilla Firefox', 'Edge', 'Google Chrome', nan,
        'Mozilla', 'Chrome', 'IE', 'InternetExplorer', 'Opera',
        'Internet Explorer', 'Safari'], dtype=object),
 array([nan, 'Desktop', 'Mobile', 'Tablet'], dtype=object))

In [0]:
train['siteid'].fillna(-999, inplace=True)
test['siteid'].fillna(-999, inplace=True)

train['browserid'].fillna("None", inplace=True)
test['browserid'].fillna("None", inplace=True)

train['devid'].fillna("None", inplace=True)
test['devid'].fillna("None", inplace=True)

In [0]:
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

In [0]:
train['tweekday'] = train['datetime'].dt.weekday
train['thour'] = train['datetime'].dt.hour
train['tminute'] = train['datetime'].dt.minute

test['tweekday'] = test['datetime'].dt.weekday
test['thour'] = test['datetime'].dt.hour
test['tminute'] = test['datetime'].dt.minute

In [0]:
train['siteid'] = train['siteid'].astype('int64')

In [16]:
train.dtypes

ID                     object
datetime       datetime64[ns]
siteid                  int64
offerid                 int64
category                int64
merchant                int64
countrycode            object
browserid              object
devid                  object
click                   int64
tweekday                int64
thour                   int64
tminute                 int64
dtype: object

In [17]:
train['devid'].unique()

array(['None', 'Desktop', 'Mobile', 'Tablet'], dtype=object)

In [0]:
cols_to_use = list(set(train.columns) - set(['ID','datetime','click']))

In [19]:
for i, c in enumerate(train[cols_to_use]): print(i,c)

0 merchant
1 countrycode
2 offerid
3 siteid
4 category
5 browserid
6 devid
7 tminute
8 tweekday
9 thour


In [20]:
train['siteid'].unique()

array([4709696, 5189467,   98480, ..., 4086584,  762989, 4284843])

In [0]:
cat_cols = cols_to_use

In [0]:
import numpy as np

In [0]:
rows = np.random.choice(train.index.values, 1000 * 1000)
strain = train.loc[rows]
sx_train = train[cols_to_use]
sy_train = train['click']

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(sx_train, sy_train, test_size=0.2)

In [25]:
!pip install catboost

     |████████████████████████████████| 64.0MB 61kB/s 


In [0]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(depth=10, iterations=10, learning_rate=0.1, eval_metric='AUC', random_seed=1)

In [27]:
model.fit(x_train, y_train, cat_features=cat_cols, eval_set=(x_test, y_test), use_best_model=True)

0:	test: 0.9633123	best: 0.9633123 (0)	total: 10s	remaining: 1m 30s
1:	test: 0.9681060	best: 0.9681060 (1)	total: 16.1s	remaining: 1m 4s
2:	test: 0.9680290	best: 0.9681060 (1)	total: 22.6s	remaining: 52.8s
3:	test: 0.9693524	best: 0.9693524 (3)	total: 28.5s	remaining: 42.8s
4:	test: 0.9699573	best: 0.9699573 (4)	total: 34.8s	remaining: 34.8s
5:	test: 0.9700139	best: 0.9700139 (5)	total: 40.6s	remaining: 27.1s
6:	test: 0.9702135	best: 0.9702135 (6)	total: 46.6s	remaining: 20s
7:	test: 0.9701862	best: 0.9702135 (6)	total: 52.4s	remaining: 13.1s
8:	test: 0.9704493	best: 0.9704493 (8)	total: 58.3s	remaining: 6.48s
9:	test: 0.9705305	best: 0.9705305 (9)	total: 1m 4s	remaining: 0us

bestTest = 0.9705305146
bestIteration = 9



In [29]:
test.dtypes

ID                     object
datetime       datetime64[ns]
siteid                float64
offerid                 int64
category                int64
merchant                int64
countrycode            object
browserid              object
devid                  object
tweekday                int64
thour                   int64
tminute                 int64
dtype: object

In [0]:
test['siteid'] = test['siteid'].astype('int64')

In [0]:
pred = model.predict_proba(test[cols_to_use])[:,1]

In [0]:
sub = pd.DataFrame({'ID':test['ID'],'click':pred})
sub.to_csv('cb_sub1.csv',index=False)

In [34]:
cat cb_sub1.csv | head

ID,click
IDFDJVI,0.17971860174719173
IDNWkTQ,0.17968417086199848
ID9pRmM,0.1804081731846483
IDHaQaj,0.17960659018305675
IDT2CrF,0.17973552993865038
IDFZS6l,0.19464720215011988
IDecQv2,0.17959668655213787
IDR9hAD,0.1945661735072258
IDzZxxC,0.18005096791790118


In [39]:
!zip sub.zip cb_sub1.csv

  adding: cb_sub1.csv (deflated 67%)
